In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.4 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 596 kB 68.1 MB/s 
     |████████████████████████████████| 6.6 MB 65.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os

# For data manipulation
import numpy as np
import pandas as pd
import warnings
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from pylab import rcParams

import torch
import torch.nn as nn

# For Transformer Models
from transformers import (
    AdamW,
    AutoTokenizer, 
    AutoModel,
    AutoConfig,
    AutoModelForSequenceClassification, 
    DataCollatorForLanguageModeling, 
    get_scheduler,
    Trainer, 
    TrainingArguments
)

from tqdm.auto import tqdm

## Загрузка данных

In [55]:
df = pd.read_csv("https://raw.githubusercontent.com/skillfactory-hackaton-team9/twitter-analyzer/main/parsed_data/twitter.csv", index_col=0)

In [56]:
df.head()

,date,author,twitter_name,text,number_of_reply,number_of_retweets,number_of_likes
0,2022-03-20T21:22:37.000Z,Фонд Подари жизнь,@podarizhizn,\n\n\n\n\n\n'фонд Подари жизнь' — Поиск в Твит...,0.0,3.0,19
1,2022-03-20T21:22:37.000Z,Masha,@arctic_lanes,"А по телеку всё идет реклама: ""Оформи ежемесяч...",29.0,63.0,335
2,2022-03-20T21:22:37.000Z,"Просто СЕРЁГА. И да, я патриот!!!",@MgNBAiiDua58SZ5,Чулпан Хаматова эмигрировала в Латвию. \nСказа...,1.0,31.0,146
3,2022-03-20T21:22:37.000Z,водка,@vodkaofsuga,Русскоязычные АРМИ пожертвовали 841 640 р. в ф...,92.0,461.0,570
4,2022-03-20T21:22:37.000Z,Compatriot Z,@cardros,"Оппа, какой интересный поворот!\nХотя, честно ...",364.0,119.0,526


In [62]:
df.shape

(3721, 7)

## Предсказания первой модели

In [37]:
MODEL_NAME = 'cointegrated/rubert-tiny-toxicity'

In [38]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
if torch.cuda.is_available():
    model.cuda()

In [39]:
def text2toxicity(text, aggregate=False):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

In [40]:
df.text.head(10)

0    \n\n\n\n\n\n'фонд Подари жизнь' — Поиск в Твит...
1    А по телеку всё идет реклама: "Оформи ежемесяч...
2    Чулпан Хаматова эмигрировала в Латвию. \nСказа...
3    Русскоязычные АРМИ пожертвовали 841 640 р. в ф...
4    Оппа, какой интересный поворот!\nХотя, честно ...
5    Фонд «Подари жизнь» Чулпан Хаматовой к началу ...
6    Почему мы все время слышим про дорогое жильё, ...
7    Чулпан Хаматову заподозрили в отмывании денег ...
8    дима с антоном поддержавшие фонд «подари жизнь...
9    \n\n\n\n\n\n'фонд Подари жизнь' — Поиск в Твит...
Name: text, dtype: object

In [41]:
sentiment = df.text.apply(text2toxicity)

In [42]:
sentiment_m1_df = pd.DataFrame.from_records(sentiment, columns=["m1_notoxic", "m1_insult", "m1_obscenity", "m1_threat", "m1_dangerous"])

In [43]:
df.join(sentiment_m1_df)

,Unnamed: 0,date,author,twitter_name,text,number_of_reply,number_of_retweets,number_of_likes,m1_notoxic,m1_insult,m1_obscenity,m1_threat,m1_dangerous
0,0,2022-03-20T21:22:37.000Z,Фонд Подари жизнь,@podarizhizn,\n\n\n\n\n\n'фонд Подари жизнь' — Поиск в Твит...,0.0,3.0,19,0.998206,0.001872,0.000105,0.000160,0.372565
1,1,2022-03-20T21:22:37.000Z,Masha,@arctic_lanes,"А по телеку всё идет реклама: ""Оформи ежемесяч...",29.0,63.0,335,0.999864,0.000180,0.000106,0.000124,0.047153
2,2,2022-03-20T21:22:37.000Z,"Просто СЕРЁГА. И да, я патриот!!!",@MgNBAiiDua58SZ5,Чулпан Хаматова эмигрировала в Латвию. \nСказа...,1.0,31.0,146,0.010285,0.981738,0.004774,0.005248,0.960458
3,3,2022-03-20T21:22:37.000Z,водка,@vodkaofsuga,Русскоязычные АРМИ пожертвовали 841 640 р. в ф...,92.0,461.0,570,0.996537,0.003471,0.000451,0.000375,0.597437
4,4,2022-03-20T21:22:37.000Z,Compatriot Z,@cardros,"Оппа, какой интересный поворот!\nХотя, честно ...",364.0,119.0,526,0.993550,0.007526,0.000244,0.000150,0.346785
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,3716,2017-05-06T16:39:36.000Z,Елена Кожевникова,@oKnAGVAoIW7tXTy,"Артем Ляшенко 3 года. Требуется лечение, Вы мо...",0.0,0.0,0,0.999836,0.000185,0.000120,0.000155,0.081041
3717,3717,2017-05-06T16:39:36.000Z,Елена Климова,@kGrqfLjWHlHY2AM,"Илья Молодкин 4 года. Требуется лечение, Вы мо...",0.0,0.0,0,0.999808,0.000209,0.000086,0.000131,0.112526
3718,3718,2017-05-06T16:39:36.000Z,Виктория Волкова,@2a5sX7wm8Wwd8Yu,"Даня Цветков 7 лет. Требуется лечение, Вы може...",0.0,0.0,0,0.999810,0.000208,0.000105,0.000130,0.150270
3719,3719,2017-05-06T16:39:36.000Z,Людмила Скоморохова,@IaqNXbciAPnRokY,"Саша Скареднова 14 лет. Требуется лечение, Вы ...",0.0,0.0,0,0.999688,0.000333,0.000134,0.000143,0.297211


In [44]:
df.shape

(3721, 8)

## Вторая модель

In [45]:
MODEL_NAME = 'SkolkovoInstitute/russian_toxicity_classifier'

In [46]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
if torch.cuda.is_available():
    model.cuda()

In [47]:
 df.text.head(10).apply(text2toxicity)

0      [0.7974871, 0.2906293]
1      [0.952932, 0.07994232]
2     [0.11824974, 0.7427905]
3     [0.6697045, 0.36554536]
4      [0.9339162, 0.1126869]
5     [0.9565279, 0.07171663]
6     [0.9519586, 0.08126402]
7     [0.88598025, 0.1778117]
8    [0.12501732, 0.71712625]
9      [0.912553, 0.15515533]
Name: text, dtype: object

In [48]:
df.loc[9,"text"]

'\n\n\n\n\n\n\'фонд Подари жизнь\' — Поиск в Твиттере / Твиттер\n\n\n\n\nФункции JavaScript недоступны.\nВ вашем браузере отключены функции JavaScript. Включите их или начните использовать поддерживаемый браузер, чтобы продолжить пользоваться сайтом twitter.com. Список поддерживаемых браузеров приведен в Справочном центре.\nСправочный центр\n\nУсловия предоставления услуг\nПолитика конфиденциальности\nПолитика в отношении файлов cookie\nImprint\nИнформация о рекламе\n      © 2022 Twitter, Inc.\n    \n\nДля просмотра горячих клавиш нажмите знак вопросаПосмотреть горячие клавишиПопулярноеПоследнееЛюдиФотографииВидеоПосмотреть новые твитыЛента поискаmaart@arcteryx74·15 окт. 2021 г.Оплатил в метро проезд лицом, метрополитен перевёл в фонд «Подари жизнь» 2 миллиарда рублей.410Мама Стифлeрa@ds107m6p·7 маяУ всего есть финансовый след.\n\n"Дебиторская задолженность фонда "Подари жизнь" (ранее известного как фонд Хаматовой) за последние годы демонстрирует прямо-таки аномальный рост более чем в 

In [49]:
sentiment2 = df.text.apply(text2toxicity)

In [50]:
sentiment_m2_df = pd.DataFrame.from_records(sentiment2, columns=["m2_neutral", "m2_toxic",])

## Объединим результаты

In [57]:
result_df = df.join(sentiment_m1_df).join(sentiment_m2_df)

In [58]:
result_df.head()

,date,author,twitter_name,text,number_of_reply,number_of_retweets,number_of_likes,m1_notoxic,m1_insult,m1_obscenity,m1_threat,m1_dangerous,m2_neutral,m2_toxic
0,2022-03-20T21:22:37.000Z,Фонд Подари жизнь,@podarizhizn,\n\n\n\n\n\n'фонд Подари жизнь' — Поиск в Твит...,0.0,3.0,19,0.998206,0.001872,0.000105,0.000160,0.372565,0.797487,0.290629
1,2022-03-20T21:22:37.000Z,Masha,@arctic_lanes,"А по телеку всё идет реклама: ""Оформи ежемесяч...",29.0,63.0,335,0.999864,0.000180,0.000106,0.000124,0.047153,0.952932,0.079942
2,2022-03-20T21:22:37.000Z,"Просто СЕРЁГА. И да, я патриот!!!",@MgNBAiiDua58SZ5,Чулпан Хаматова эмигрировала в Латвию. \nСказа...,1.0,31.0,146,0.010285,0.981738,0.004774,0.005248,0.960458,0.118250,0.742791
3,2022-03-20T21:22:37.000Z,водка,@vodkaofsuga,Русскоязычные АРМИ пожертвовали 841 640 р. в ф...,92.0,461.0,570,0.996537,0.003471,0.000451,0.000375,0.597437,0.669704,0.365545
4,2022-03-20T21:22:37.000Z,Compatriot Z,@cardros,"Оппа, какой интересный поворот!\nХотя, честно ...",364.0,119.0,526,0.993550,0.007526,0.000244,0.000150,0.346785,0.933916,0.112687


In [59]:
result_df.shape

(3721, 14)

In [60]:
result_df.to_csv("sentiment_from_2_models.csv")